In [ ]:
import random
import numpy as np
import threading
from PIL import Image
import matplotlib.pyplot as plt
from time import time
import gc
import mxnet as mx
from mxnet import nd
from mxnet import gluon
from mxnet import init
from mxnet import autograd
%matplotlib inline
import shutil

In [ ]:
import os
os.environ['MXNET_GPU_MEM_POOL_RESERVE'] = '25'

In [ ]:
random.seed(1)
np.random.seed(1)
mx.random.seed(1)

In [ ]:
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.array([0], ctx=ctx)
    except:
        ctx = mx.cpu()
    return ctx
ctx = try_gpu()

# Read Data

In [ ]:
def read_vocab(path):
    filelist = ['type.dat','color.dat','material.dat','model.dat','pattern.dat','style.dat','position.dat']
    word_vocab = {}
    index_vocab = {}
    index = 0
    for file in filelist:
        with open(path+file,'r',encoding='utf-8') as f:
            content = f.readlines()
        for line in content:
            line = line[:-1].split()
            for word in line:
                word_vocab[word] = index
            index_vocab[index] = line
            index += 1
        del content
    return word_vocab, index_vocab

In [ ]:
def read_itemlist(path, word_vocab):
    with open(path,'r',encoding='utf-8') as f:
        content = f.readlines()
    itemlist = {}
    for line in content:
        line = line[:-1].split('\t')
        item = line[0]
        description = line[1]
        combination = set(line[2].split('|'))
        itemlist[item] = {'description':description,'combination':combination}
        del line
    del content
    return itemlist

In [ ]:
def read_descriptionlist(path, word_vocab, vocab_size):
    with open(path,'r',encoding='utf-8') as f:
        content = f.readlines()
    descriptionlist = {}
    for line in content:
        line = line[:-1].split('\t')
        id = line[0]
        text = line[1]
        items = set(line[2].split('|'))
        vector = np.zeros(vocab_size)
        for word in text.split(' '):
            if word in word_vocab:
                vector[word_vocab[word]] = 1
        descriptionlist[id] = {'vector':vector,'text':text,'items':items}
        del line,vector
    del content
    return descriptionlist

In [ ]:
class myThread (threading.Thread):
    def __init__(self, imglist, img_idxs):
        threading.Thread.__init__(self)
        self.imglist = imglist
        self.img_idxs = img_idxs
    def run(self): 
        for img_idx in self.img_idxs:
            img = Image.open('img/'+img_idx+'.jpg')
            img = np.array(img)
            img = img/255.0
            img = img.transpose([2,0,1])
            self.imglist[img_idx] = img
            del img

In [ ]:
path = 'dataset/vocab/'
word_vocab,index_vocab = read_vocab(path)
len(index_vocab)

In [ ]:
path = 'dataset/toplist.dat'
toplist = read_itemlist(path, word_vocab)
alltops = list(toplist.keys())

In [ ]:
path = 'dataset/bottomlist.dat'
bottomlist = read_itemlist(path, word_vocab)
allbottoms = list(bottomlist.keys())

In [ ]:
path = 'dataset/descriptionlist.dat'
descriptionlist = read_descriptionlist(path, word_vocab, len(index_vocab))

In [ ]:
imglist = {}
tops = random.sample(alltops,50000)
bottoms = random.sample(allbottoms,50000)
threads = []
thread1 = myThread(imglist,tops[:len(tops)//2])
thread2 = myThread(imglist,tops[len(tops)//2:])
thread3 = myThread(imglist,bottoms[:len(bottoms)//2])
thread4 = myThread(imglist,bottoms[len(bottoms)//2:])
thread1.start()
thread2.start()
thread3.start()
thread4.start()
threads.append(thread1)
threads.append(thread2)
threads.append(thread3)
threads.append(thread4)
for t in threads:
    t.join()
print(len(imglist)==len(tops)+len(bottoms))
del threads,tops,bottoms

In [ ]:
outfitlist = {}
with open('dataset/outfitlist.dat','r',encoding='utf-8') as f:
    content = f.readlines()
for line in content:
    line = line[:-1].split('\t')
    top = line[0]
    description = line[1]
    bottoms = line[2].split('|')
    outfitlist[(top,description)] = set(bottoms)
    del line
del content

In [ ]:
train_data = []
with open('dataset/traindata.dat','r',encoding='utf-8') as f:
    content = f.readlines()
for line in content:
    line = line[:-1].split('\t')
    top = line[0]
    description = line[1]
    bottoms = line[2].split('|')
    for bottom in bottoms:
        train_data.append((top,description,bottom))
    del line
del content

In [ ]:
valid_data = {}
with open('dataset/validdata.dat','r',encoding='utf-8') as f:
    content = f.readlines()
for line in content:
    line = line[:-1].split('\t')
    top = line[0]
    description = line[1]
    bottom = line[2]
    negatives = line[3].split('|')
    valid_data[(top,description,bottom)] = negatives
    del line
del content

In [ ]:
test_data = {}
with open('dataset/testdata.dat','r',encoding='utf-8') as f:
    content = f.readlines()
for line in content:
    line = line[:-1].split('\t')
    top = line[0]
    description = line[1]
    bottom = line[2]
    negatives = line[3].split('|')
    test_data[(top,description,bottom)] = negatives
    del line
del content

In [ ]:
gc.collect()

# Build Model

In [ ]:
class SRResNetBlock(gluon.nn.Block):
    def __init__(self, channels, same_shape=True, **kwargs):
        super(SRResNetBlock, self).__init__(**kwargs)
        self.same_shape = same_shape
        strides = 1 if same_shape else 2
        self.conv1 = gluon.nn.Conv2D(channels, kernel_size=3, padding=1, strides=strides)
        self.conv2 = gluon.nn.Conv2D(channels, kernel_size=3, padding=1)
        if not same_shape:
            self.conv3 = gluon.nn.Conv2D(channels, kernel_size=1, strides=strides)
        self.bn1 = gluon.nn.BatchNorm()
        self.bn2 = gluon.nn.BatchNorm()
    def forward(self, x):
        out = nd.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if not self.same_shape:
            x = self.conv3(x)
        return out+x

In [ ]:
class farm_top_encoder(gluon.nn.Block):
    def __init__(self, **kwargs):
        super(farm_top_encoder, self).__init__(**kwargs) 
        self.convs = gluon.nn.Sequential()
        self.convs.add(
            gluon.nn.Conv2D(channels=64, kernel_size=4, strides=2, padding=1),#64*64*64
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2D(channels=128, kernel_size=4, strides=2, padding=1),#128*32*32
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2D(channels=256, kernel_size=4, strides=2, padding=1),#256*16*16
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2D(channels=512, kernel_size=4, strides=2, padding=1),#512*8*8
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2D(channels=512, kernel_size=4, strides=2, padding=1),#512*4*4
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2D(channels=1024, kernel_size=4, strides=1, padding=0),#1024*1*1
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Flatten(),#1024
            gluon.nn.Dropout(0.5),
            gluon.nn.Dense(100, activation='sigmoid')
        )
    def forward(self, img):
        enc = self.convs(img)
        return enc

In [ ]:
class farm_bottom_encoder(gluon.nn.Block):
    def __init__(self, **kwargs):
        super(farm_bottom_encoder, self).__init__(**kwargs) 
        self.convs_1 = gluon.nn.Sequential()
        self.convs_1.add(
            gluon.nn.Conv2D(channels=64, kernel_size=4, strides=2, padding=1),#64*64*64
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2D(channels=128, kernel_size=4, strides=2, padding=1),#128*32*32
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2D(channels=256, kernel_size=4, strides=2, padding=1),#256*16*16
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2D(channels=512, kernel_size=4, strides=2, padding=1),#512*8*8
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu')
        )
        self.feature_3 = gluon.nn.Sequential()
        self.feature_3.add(
            gluon.nn.GlobalAvgPool2D(),#[batch_size,512,1,1]
            gluon.nn.Flatten(),#[batch_size,512]
            gluon.nn.Dense(100, activation='sigmoid') 
        )
        self.convs_2 = gluon.nn.Sequential()
        self.convs_2.add(
            gluon.nn.Conv2D(channels=512, kernel_size=4, strides=2, padding=1),#512*4*4
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu')
        )
        self.feature_2 = gluon.nn.Sequential()
        self.feature_2.add(
            gluon.nn.GlobalAvgPool2D(),#[batch_size,512,1,1]
            gluon.nn.Flatten(),#[batch_size,512]
            gluon.nn.Dense(100, activation='sigmoid') 
        )
        self.convs_3 = gluon.nn.Sequential()
        self.convs_3.add(
            gluon.nn.Conv2D(channels=1024, kernel_size=4, strides=1, padding=0),#1024*1*1
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Flatten()#[batch_size,1024]
        )
        self.feature_1 = gluon.nn.Sequential()
        self.feature_1.add(
            gluon.nn.Dense(100, activation='sigmoid') 
        )      
        self.dropout = gluon.nn.Dropout(0.5)
        self.dense = gluon.nn.Dense(100, activation='sigmoid')
    def forward(self, img):
        enc = self.convs_1(img)
        feature_3 = self.feature_3(enc)
        enc = self.convs_2(enc)
        feature_2 = self.feature_2(enc)
        enc = self.convs_3(enc)
        feature_1 = self.feature_1(enc)     
        enc = self.dropout(enc)
        enc = self.dense(enc)
        return enc, feature_1, feature_2, feature_3

In [ ]:
class farm_transformer(gluon.nn.Block):
    def __init__(self, **kwargs):
        super(farm_transformer, self).__init__(**kwargs)     
        self.dense_1 = gluon.nn.Dense(100)
        self.dense_2 = gluon.nn.Dense(100)
    def forward(self, enc_x, enc_c):
        con = nd.concat(enc_x, enc_c, dim=1)
        z_mean = self.dense_1(con)
        z_log_var = self.dense_2(con)
        epsilon = nd.random_normal(loc=0, scale=1, shape=z_mean.shape, ctx=ctx)
        z = z_mean+nd.exp(0.5*z_log_var)*epsilon
        return z,z_mean,z_log_var

In [ ]:
class farm_generator(gluon.nn.Block):
    def __init__(self, **kwargs):
        super(farm_generator, self).__init__(**kwargs)
        self.seq = gluon.nn.Sequential()
        self.seq.add(
            gluon.nn.Dense(1024),
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu')
        )
        self.feature_1 = gluon.nn.Sequential()
        self.feature_1.add(
            gluon.nn.Dense(100, activation='sigmoid')
        )
        self.convs_1 = gluon.nn.Sequential()
        self.convs_1.add(
            gluon.nn.Conv2DTranspose(channels=512, kernel_size=4, strides=1, padding=0),#512*4*4
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu')
        )
        self.feature_2 = gluon.nn.Sequential()
        self.feature_2.add(
            gluon.nn.GlobalAvgPool2D(),#[batch_size,512,1,1]
            gluon.nn.Flatten(),#[batch_size,512]
            gluon.nn.Dense(100, activation='sigmoid')
        )
        self.convs_2 = gluon.nn.Sequential()
        self.convs_2.add(
            gluon.nn.Conv2DTranspose(channels=512, kernel_size=4, strides=2, padding=1),#512*8*8
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu')
        )
        self.feature_3 = gluon.nn.Sequential()
        self.feature_3.add(
            gluon.nn.GlobalAvgPool2D(),#[batch_size,512,1,1]
            gluon.nn.Flatten(),#[batch_size,512]
            gluon.nn.Dense(100, activation='sigmoid')
        )
        self.convs_3 = gluon.nn.Sequential()
        self.convs_3.add(
            gluon.nn.Conv2DTranspose(channels=256, kernel_size=4, strides=2, padding=1),#256*16*16
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2DTranspose(channels=128, kernel_size=4, strides=2, padding=1),#128*32*32
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2DTranspose(channels=64, kernel_size=4, strides=2, padding=1),#64*64*64
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu'),
            gluon.nn.Conv2DTranspose(channels=3, kernel_size=4, strides=2, padding=1),#3*128*128
            gluon.nn.Activation('sigmoid')
        )
        self.convs_4 = gluon.nn.Sequential()
        self.convs_4.add(
            gluon.nn.Conv2D(channels=32, kernel_size=3, strides=1, padding=1),#32*128*128
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu')
        )
        self.convs_5 = gluon.nn.Sequential()
        self.convs_5.add(
            SRResNetBlock(32),
            SRResNetBlock(32),
            SRResNetBlock(32),
            gluon.nn.Conv2D(channels=32, kernel_size=3, strides=1, padding=1),#32*128*128
            gluon.nn.BatchNorm(),
            gluon.nn.Activation('relu')
        )
        self.convs_6 = gluon.nn.Sequential()
        self.convs_6.add(
            gluon.nn.Conv2D(channels=3, kernel_size=1, strides=1, padding=0),#3*128*128
            gluon.nn.Activation('sigmoid')
        )
    def forward(self, z, enc_x, enc_c):
        dec = nd.concat(z, enc_x, enc_c, dim=1)
        dec = self.seq(dec)#[batch_size,1024]  
        feature_1 = self.feature_1(dec)
        dec = dec.reshape(shape=(dec.shape[0], 1024, 1, 1))#1024*1*1
        dec = self.convs_1(dec)
        feature_2 = self.feature_2(dec)
        dec = self.convs_2(dec)
        feature_3 = self.feature_3(dec)
        y_rec_low = self.convs_3(dec)
        low_to_high_1 = self.convs_4(y_rec_low)
        low_to_high_2 = self.convs_5(low_to_high_1)
        y_rec_high = self.convs_6(low_to_high_1+low_to_high_2)
        return y_rec_low, y_rec_high, feature_1, feature_2, feature_3

In [ ]:
class farm_recommender(gluon.nn.Block):
    def __init__(self, **kwargs):
        super(farm_recommender, self).__init__(**kwargs)
    def forward(self, enc_x, enc_c, enc_yp, enc_yn, feature_1_yp, feature_2_yp, feature_3_yp, feature_1_yn, feature_2_yn, feature_3_yn, feature_1_yg, feature_2_yg, feature_3_yg):
        score_yp = nd.sum(enc_yp*enc_x, axis=1)+nd.sum(enc_yp*enc_c, axis=1)+nd.sum(feature_1_yp*feature_1_yg, axis=1)+nd.sum(feature_2_yp*feature_2_yg, axis=1)+nd.sum(feature_3_yp*feature_3_yg, axis=1)
        score_yn = nd.sum(enc_yn*enc_x, axis=1)+nd.sum(enc_yn*enc_c, axis=1)+nd.sum(feature_1_yn*feature_1_yg, axis=1)+nd.sum(feature_2_yn*feature_2_yg, axis=1)+nd.sum(feature_3_yn*feature_3_yg, axis=1)
        difference_pn = score_yp-score_yn
        return difference_pn

In [ ]:
class farm(gluon.nn.Block):
    def __init__(self, **kwargs):
        super(farm, self).__init__(**kwargs)
        self.description_embedding = gluon.nn.Dense(100, activation='sigmoid')
        self.top_encoder = farm_top_encoder()
        self.bottom_encoder = farm_bottom_encoder()
        self.transformer = farm_transformer()
        self.generator = farm_generator()
        self.recommender = farm_recommender()
    def forward(self, top_img, bottom_img, negative_img, bottom_description, bottom_description_drop, phase):
        enc_c = self.description_embedding(bottom_description)
        enc_c_drop = self.description_embedding(bottom_description_drop)
        enc_x = self.top_encoder(top_img)
        enc_yp, feature_1_yp, feature_2_yp, feature_3_yp = self.bottom_encoder(bottom_img)
        enc_yn, feature_1_yn, feature_2_yn, feature_3_yn = self.bottom_encoder(negative_img)
        z,z_mean,z_log_var = self.transformer(enc_x, enc_c_drop)
        if phase == 'train':
            y_rec_low, y_rec_high, feature_1_yg, feature_2_yg, feature_3_yg = self.generator(z, enc_x, enc_c_drop)
        if phase == 'test':
            y_rec_low, y_rec_high, feature_1_yg, feature_2_yg, feature_3_yg = self.generator(z_mean, enc_x, enc_c_drop)
        difference_pn = self.recommender(enc_x, enc_c, enc_yp, enc_yn, feature_1_yp, feature_2_yp, feature_3_yp, feature_1_yn, feature_2_yn, feature_3_yn, feature_1_yg, feature_2_yg, feature_3_yg) 
        return y_rec_low, y_rec_high, difference_pn, z_mean, z_log_var#, feature_1_yp, feature_2_yp, feature_3_yp, feature_1_yn, feature_2_yn, feature_3_yn, feature_1_yg, feature_2_yg, feature_3_yg

In [ ]:
l2loss = gluon.loss.L2Loss()
l1loss = gluon.loss.L1Loss()
def loss(y, y_rec_low, y_rec_high, difference_pn, z_mean, z_log_var, l2loss, l1loss):
    rec_loss = nd.sum(y.shape[1]*y.shape[2]*y.shape[3]*l2loss(y,y_rec_low))+nd.sum(y.shape[1]*y.shape[2]*y.shape[3]*l1loss(y,y_rec_high))  
    kl_loss = nd.sum(-0.5*nd.sum(1+z_log_var-nd.square(z_mean)-nd.exp(z_log_var), axis=-1))
    bpr_loss = nd.sum(-nd.log(nd.sigmoid(difference_pn)))
    loss = rec_loss+kl_loss+bpr_loss
    return loss, bpr_loss, rec_loss+kl_loss

In [ ]:
def get_img(img_idx,imglist):
    if img_idx in imglist:
        return imglist[img_idx]
    else:
        imglist.popitem()
        img = Image.open('img/'+img_idx+'.jpg')
        img = np.array(img)
        img = img/255.0
        img = img.transpose([2,0,1])
        imglist[img_idx] = img
        del img
        return imglist[img_idx]

In [ ]:
def negative_sample(top, description, outfitlist, allbottoms):
    bottoms = outfitlist[(top, description)]
    while True:
        bottom = random.sample(allbottoms,1)[0]
        if bottom not in bottoms:
            return bottom

In [ ]:
def description_dropout(description, keep_rate):
    if keep_rate < 1.0:
        filter = np.random.uniform(low=0.0, high=1.0, size=description.shape) < keep_rate
        return description*filter
    else:
        return description

In [ ]:
def batch_to_input(batch, imglist, descriptionlist, outfitlist, allbottoms, ctx, keep_rate):
    top_img = []
    bottom_img = []
    bottom_description = []
    bottom_description_drop = []
    negative_img = []
    for instance in batch:
        top_img.append(get_img(instance[0], imglist))
        bottom_img.append(get_img(instance[2], imglist))
        bottom_description.append(descriptionlist[instance[1]]['vector'])
        bottom_description_drop.append(description_dropout(descriptionlist[instance[1]]['vector'], keep_rate))
        negative_img.append(get_img(negative_sample(instance[0], instance[1], outfitlist, allbottoms), imglist))
    top_img_mx = nd.array(top_img, ctx=ctx)
    bottom_img_mx = nd.array(bottom_img, ctx=ctx)
    bottom_description_mx = nd.array(bottom_description, ctx=ctx)
    bottom_description_drop_mx = nd.array(bottom_description_drop, ctx=ctx)
    negative_img_mx = nd.array(negative_img, ctx=ctx)
    del top_img, bottom_img, bottom_description, bottom_description_drop, negative_img
    return top_img_mx, bottom_img_mx, bottom_description_mx, bottom_description_drop_mx, negative_img_mx 

In [ ]:
def get_batches(data, imglist, descriptionlist, outfitlist, allbottoms, batch_size, ctx, keep_rate):
    random.shuffle(data)
    for batch_i in range(0,len(data)//batch_size+1):
        start_i = batch_i*batch_size
        batch = data[start_i:start_i+batch_size]
        yield batch_to_input(batch, imglist, descriptionlist, outfitlist, allbottoms, ctx, keep_rate)

In [ ]:
def evaluation_batch_to_input(batch, imglist, descriptionlist, ctx):
    top_img = []
    bottom_img = []
    bottom_description = []
    negative_img = []
    for instance in batch:
        top_img.append(get_img(instance[0], imglist))
        bottom_img.append(get_img(instance[2], imglist))
        bottom_description.append(descriptionlist[instance[1]]['vector'])
        negative_img.append(get_img(instance[3], imglist))
    top_img_mx = nd.array(top_img, ctx=ctx)
    bottom_img_mx = nd.array(bottom_img, ctx=ctx)
    bottom_description_mx = nd.array(bottom_description, ctx=ctx)
    negative_img_mx = nd.array(negative_img, ctx=ctx)
    del top_img, bottom_img, bottom_description, negative_img
    return top_img_mx, bottom_img_mx, bottom_description_mx, negative_img_mx 

In [ ]:
def get_evaluation_batches(top, description, bottom, negatives, imglist, descriptionlist, batch_size, ctx):
    data = []
    for negative in negatives:
        data.append([top,description,bottom,negative])
    for batch_i in range(0,len(data)//batch_size+1):
        start_i = batch_i*batch_size
        batch = data[start_i:start_i+batch_size]            
        yield evaluation_batch_to_input(batch, imglist, descriptionlist, ctx)        

In [ ]:
net = farm()
net.initialize(ctx=ctx,init=init.Xavier())

In [ ]:
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 0.001, 'clip_gradient': 5})

In [ ]:
def train(train_data, valid_data, imglist, descriptionlist, outfitlist, allbottoms, net, loss, l2loss, l1loss, trainer, ctx, batch_size, keep_rate, num_epochs, print_batches=None):
    """Train a network"""
    print("Start training on ", ctx)
    for epoch in range(num_epochs):
        train_rec_loss, train_gen_loss, valid_auc, valid_gen_loss, n = 0.0, 0.0, 0.0, 0.0, 0.0
        start = time()
        for i, (top_img, bottom_img, bottom_description, bottom_description_drop, negative_img) in enumerate(get_batches(train_data, imglist, descriptionlist, outfitlist, allbottoms, batch_size, ctx, keep_rate)):
            with autograd.record():
                y_rec_low, y_rec_high, difference_pn, z_mean, z_log_var = net(top_img, bottom_img, negative_img, bottom_description, bottom_description_drop, 'train')
                l, l_rec, l_gen = loss(bottom_img, y_rec_low, y_rec_high, difference_pn, z_mean, z_log_var, l2loss, l1loss)
            l.backward()
            trainer.step(batch_size)
            train_rec_loss += l_rec.asscalar()
            train_gen_loss += l_gen.asscalar()
            n += batch_size
            del top_img, bottom_img, bottom_description, bottom_description_drop, negative_img
            if print_batches and (i+1) % print_batches == 0:
                print("Batch %d. Train_Rec_Loss: %f, Train_Gen_Loss: %f" % (
                    n, train_rec_loss/n, train_gen_loss/n
                )) 
                gc.collect()
        net.collect_params().save('checkpoint/farm_params_'+str(epoch)+'.dat')
        for (top, description, bottom), negatives in valid_data.items():
            auc = []
            for i, (top_img, bottom_img, bottom_description, negative_img) in enumerate(get_evaluation_batches(top, description, bottom, negatives, imglist, descriptionlist, batch_size, ctx)):  
                _, y_rec, difference_pn, _, _ = net(top_img, bottom_img, negative_img, bottom_description, bottom_description, 'test')
                l = nd.mean(y_rec.shape[1]*y_rec.shape[2]*y_rec.shape[3]*l2loss(bottom_img,y_rec)) 
                auc += list(difference_pn.asnumpy())
                del top_img, bottom_img, bottom_description, negative_img
            valid_gen_loss += l.asscalar()
            valid_auc += 1-(np.array(auc) <= 0).sum()/len(auc)
        print("Epoch %d. Train_Rec_Loss: %.3f, Train_Gen_Loss: %.3f, Valid_AUC: %.3f, Valid_Gen_Loss: %.3f, Time %.1f sec" % (
            epoch, train_rec_loss/len(train_data), train_gen_loss/len(train_data), valid_auc/len(valid_data), valid_gen_loss/len(valid_data), time()-start
        ))
        gc.collect()

In [ ]:

train(train_data, valid_data, imglist, descriptionlist, outfitlist, allbottoms, net, loss, l2loss, l1loss, trainer, ctx, 64, 1.0, 50, print_batches=100)
#train(train_data, valid_data, imglist, descriptionlist, outfitlist, alltops, net, loss, l2loss, l1loss, trainer, ctx, 64, 1.0, 50, print_batches=100)#for top recommendation

In [ ]:
len(imglist)

# Test Model

In [ ]:
net.collect_params().load('checkpoint/farm_params_'+str(epoch)+'.dat', ctx=ctx)

In [ ]:
def test(test_data, imglist, descriptionlist, bottomlist, net, l1loss, ctx, batch_size, output=False):   
    test_auc, test_mrr, test_map, test_gen_loss = 0.0, 0.0, 0.0, 0.0
    k = 0
    for (top, description, bottom), negatives in test_data.items():
        auc = []
        for i, (top_img, bottom_img, bottom_description, negative_img) in enumerate(get_evaluation_batches(top, description, bottom, negatives, imglist, descriptionlist, batch_size, ctx)):  
            _, y_rec, difference_pn, _, _ = net(top_img, bottom_img, negative_img, bottom_description, bottom_description, 'test')
            l = nd.mean(y_rec.shape[1]*y_rec.shape[2]*y_rec.shape[3]*l2loss(bottom_img,y_rec))              
            auc += list(difference_pn.asnumpy())
            del top_img, bottom_img, bottom_description, negative_img
        test_gen_loss += l.asscalar()
        test_auc += 1-(np.array(auc) <= 0).sum()/len(auc)
        test_mrr += 1/((np.array(auc) <= 0).sum()+1)
        test_map += 1/((np.array(auc) <= 0).sum()+1)
        if output:
            record = {}
            j = 0
            for neg in negatives:
                record[neg] = auc[j]
                j += 1
            record[bottom] = 0.0
            record = sorted(record.items(),key=lambda item:item[1],reverse=False)
            os.makedirs('output/'+str(k))
            with open('output/'+str(k)+'/text.txt','w',encoding='utf-8') as f:
                shutil.copyfile('img/'+top+'.jpg','output/'+str(k)+'/'+top+'.jpg')
                f.write(top+'\t'+descriptionlist[description]['text']+'\n')
                j = 0
                while j < 10:
                    img = record[j][0]
                    if img is not bottom:
                        shutil.copyfile('img/'+img+'.jpg','output/'+str(k)+'/'+str(j)+'.jpg')
                    else:
                        shutil.copyfile('img/'+img+'.jpg','output/'+str(k)+'/'+str(j)+'_.jpg')
                    des = bottomlist[img]['description']
                    f.write(img+'\t'+descriptionlist[des]['text']+'\n')
                    j += 1
            y_rec = y_rec[0].transpose([1,2,0]).asnumpy()
            plt.imsave('output/'+str(k)+'/rec.jpg',y_rec)
            del record
        k += 1
    print("Test_AUC: %.3f, Test_MRR: %.3f, Test_MAP: %.3f, Test_Gen_Loss: %.3f" % (
        test_auc/len(test_data), test_mrr/len(test_data), test_map/len(test_data), test_gen_loss/len(test_data)
    ))

In [ ]:
test(test_data, imglist, descriptionlist, bottomlist, net, l2loss, ctx, 64, True)
#test(test_data, imglist, descriptionlist, toplist, net, l2loss, ctx, 64, True)#for top recommendation